In [ ]:

from bs4 import BeautifulSoup
import requests

In [ ]:
BASE_URL = "https://www.migros.ch/en"
response = requests.get(BASE_URL)
response.status_code  # Should return 200 if the request was successful


In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from urllib.parse import quote_plus
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Retrieve username and password from environment variables
MONGODB_USERNAME = os.getenv('MONGO_USERNAME')
MONGODB_PASSWORD = quote_plus(os.getenv('MONGO_PASSWORD'))
MONGO_DB_NAME = os.getenv('MONGO_DB_NAME')


# Construct the MongoDB URI
MONGODB_URI = f"mongodb+srv://{MONGODB_USERNAME}:{MONGODB_PASSWORD}@{MONGO_DB_NAME}.drzjb.mongodb.net/?retryWrites=true&w=majority&appName={MONGO_DB_NAME}"

# Create a new client and connect to the server
client = MongoClient(MONGODB_URI, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    db = client[MONGO_DB_NAME] 
    collections = db.list_collection_names()

    print("Existing collections:", collections)
except Exception as e:
    print(e)


In [ ]:
# MongoDB setup
from services.mongo_service import MongoService
from migros_scraper import MigrosScraper



mongo_service = MongoService(uri=MONGODB_URI, db_name=MONGO_DB_NAME)

# Initialize the scraper
scraper = MigrosScraper(mongo_service=mongo_service)

try:
    # Scrape level 1 categories
    base_categories = scraper.get_base_categories()
    print("Base Categories:", base_categories)

    # Step 2: For each base category, scrape it
    scraper.scrape_categories_from_base()
    print(scraper.driver.requests)
except Exception as e:
    print(e)
    scraper.close()
    mongo_service.close()

# Close the scraper
scraper.close()
mongo_service.close()